# Browser Automation Lecture Notes
Date: 2023-07-18

We'll use this notebook to work through some examples and showcase some essential functions in Selenium.

Rather than basic Selenium, we'll use Selenium Wire, which can be used to intercept API calls/network requests.

In [1]:
# !pip install selenium selenium-wire chromedriver-binary-auto

In [2]:
import os
import random
import time

from seleniumwire import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.common.exceptions import (
    MoveTargetOutOfBoundsException,
    TimeoutException,
    WebDriverException,
)

import chromedriver_binary

### Open browser, visit Amazon and perform a search

In [3]:
# Open a browser session
driver = webdriver.Chrome()

# Visit a page
driver.get('https://amazon.com')

# Find the search box
search_box = driver.find_element(By.XPATH, './/input[@aria-label="Search Amazon"]')

# Write something to search in the search box
search_box.send_keys('womens shirt')

# Press enter (in general)
ActionChains(driver).send_keys(Keys.ENTER).perform()

### Parsing

**Note**: Leon recommends not using Selenium for parsing, but only for BA. Save down the page source and then parse the saved results in BeautifulSoup, lxml, or whatever parsing software you prefer.

In [ ]:
# list all the product elements
xpath = './/div[contains(@cel_widget_id, "MAIN-SEARCH_RESULTS-")]'
product_tiles = driver.find_elements(By.XPATH, xpath)

# print the brand name of each product
for product in product_tiles:
    print(product.find_element(By.TAG_NAME, 'h2').text)

### Running javascript

Highlight merchendise from ads in red

In [15]:
ads = driver.find_elements(
    By.XPATH, 
    # you can use XPATH to specify the attributes of the children of the node you want...
    './/div[@data-asin and .//a[@aria-label="View Sponsored information or leave ad feedback"]]'
)

print(len(ads))

color='red'
js = f"arguments[0].setAttribute('style','background-color: {color} !important; transition: all 0.5s linear;')"
for elem in ads:
    driver.execute_script(js, elem)

11


### Get the positions of the elements in the page

In [16]:
import pandas as pd

In [17]:
height = driver.execute_script("return document.body.scrollHeight")
height

17577

Get the coorindates and size of each element using the `rect` function.

In [18]:
ad_metadata = [elem.rect for elem in ads if elem.is_displayed()] #only visible elements
ad_metadata

[{'height': 689, 'width': 293, 'x': 602, 'y': 516.96875},
 {'height': 689, 'width': 293, 'x': 895, 'y': 516.96875},
 {'height': 651, 'width': 293, 'x': 309, 'y': 1205.671875},
 {'height': 436, 'width': 879, 'x': 309, 'y': 2565.078125},
 {'height': 719, 'width': 293, 'x': 895, 'y': 3041.03125},
 {'height': 681, 'width': 293, 'x': 309, 'y': 3759.734375},
 {'height': 681, 'width': 293, 'x': 602, 'y': 3759.734375},
 {'height': 692, 'width': 293, 'x': 309, 'y': 5040.140625},
 {'height': 692, 'width': 293, 'x': 602, 'y': 5040.140625},
 {'height': 692, 'width': 293, 'x': 895, 'y': 5040.140625},
 {'height': 640, 'width': 293, 'x': 309, 'y': 5731.84375}]

In [19]:
df = pd.DataFrame(ad_metadata)
df['how_far_down'] = df['y'] / height
df

,height,width,x,y,how_far_down
0,689,293,602,516.968750,0.029412
1,689,293,895,516.968750,0.029412
2,651,293,309,1205.671875,0.068594
3,436,879,309,2565.078125,0.145934
4,719,293,895,3041.031250,0.173012
5,681,293,309,3759.734375,0.213901
6,681,293,602,3759.734375,0.213901
7,692,293,309,5040.140625,0.286746
8,692,293,602,5040.140625,0.286746
9,692,293,895,5040.140625,0.286746


In [20]:
df['how_far_down'].value_counts()

how_far_down
0.286746    3
0.029412    2
0.213901    2
0.068594    1
0.145934    1
0.173012    1
0.326099    1
Name: count, dtype: int64

### Save receipts

In [21]:
# how to save what the emulator sees
source = driver.page_source
with open('data/amazon_selenium_women_sshirts.html', 'w') as f:
    f.write(source)

In [22]:
# just what's visible in the screen
driver.save_screenshot('data/amazon_selenium_test.png')

True

There's are ways to do a full screen screenshot, but none of my function seem too work. Can you take a full-screenshot?

### Parsing the results however you like
For me it means using lxml, but you can do this same thing in BeautifulSoup, and I encourage you do so...

In [ ]:
# !pip install lxml

In [ ]:
from lxml import etree

In [ ]:
dom = etree.HTML(open('data/amazon_selenium_women_sshirts.html').read())
# dom = etree.HTML(driver.page_source)

In [ ]:
product_metadata = []
for result in dom.xpath('.//div[contains(@cel_widget_id, "MAIN-SEARCH_RESULTS")]'):
    # this is where you can parse as many fields as you like.
    brand, product_name = result.xpath('.//h2//text()')[:2]
    product_metadata.append({
        'brand': brand,
        'product_name': product_name
    })

In [ ]:
pd.DataFrame(product_metadata)

### SheWin, She Spin... or rotate
1. Find all products
2. filter to those with brand == SHEWIN
3. Find the image
    - Inject Javascript to make it spin.

In [ ]:
def spin(driver, elem):
    """
    Injects a style attribute to rotate `elem` 180 degrees.
    """
    style = f"transform: rotate(180deg) !important; "
    driver.execute_script(
            f"arguments[0].setAttribute('style','{style}')", elem
    )

In [ ]:
for elem in ads:
    spin(driver, elem)
    # stain(driver, elem)

In [ ]:
brands_to_spin = [
    'SHEWIN',
    'Blooming Jelly'
]

In [ ]:
product_tiles = driver.find_elements(
    By.XPATH, 
    './/div[contains(@cel_widget_id, "MAIN-SEARCH_RESULTS-")]' # the contains syntac allows a substring match
)
len(product_tiles)

## Here's the bountry
Ultimately, I want every product image from specific brands (of y0ur choosing) to rotate continuously.
You need a full screenshot or a video is fine, too.

Also, make sure to save the results before you parse them.

In [ ]:
for product in product_tiles:
    # get the brand name...
    brand_name = product.find_element(
        By.TAG_NAME, 'h2'
        # By.XPATH, './/h2'

    ).text
    
    # check if brand is in list
    if brand_name in brands_to_spin:
        print(product.text)
        # find the image TK
        spin(driver, product)